<a href="https://colab.research.google.com/github/SaiDheerajPeketi/Major-Project-CSC06-Alt/blob/main/Major_Project_Reboot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV, ParameterGrid
import joblib
import os
from datetime import datetime

In [ ]:
# Create directories for saving models and results
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

In [ ]:
# Load the data
df = pd.read_csv('/content/drive/MyDrive/Dataset/raw_combined_data.csv')

In [ ]:
# Data Exploration
print("Data shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

print("\nData types:")
print(df.dtypes)

print("\nMissing values:")
print(df.isnull().sum())

print("\nDescriptive statistics:")
print(df.describe())

In [ ]:
# Convert timestamp to datetime
if 'Timestamp' in df.columns and not df['Timestamp'].isna().all():
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

In [ ]:
# Define function to fill missing values
def fill_missing_values(df):
    # For SpO2 columns - fill with median as it's a vital sign with normal range
    spo2_cols = [col for col in df.columns if 'SpO2' in col]
    for col in spo2_cols:
        if df[col].isna().sum() > 0 and df[col].notna().sum() > 0:
            df[col] = df[col].fillna(df[col].median())

    # For heart rate columns - fill with median
    hr_cols = [col for col in df.columns if '_HR' in col]
    for col in hr_cols:
        if df[col].isna().sum() > 0 and df[col].notna().sum() > 0:
            df[col] = df[col].fillna(df[col].median())

    # For PI (Perfusion Index) - fill with median
    pi_cols = [col for col in df.columns if '_PI' in col]
    for col in pi_cols:
        if df[col].isna().sum() > 0 and df[col].notna().sum() > 0:
            df[col] = df[col].fillna(df[col].median())

    # For ETCO2, ETO2, ScalcO2, RR - fill with median
    other_cols = ['ETCO2', 'ETO2', 'ScalcO2', 'RR']
    for col in other_cols:
        if col in df.columns and df[col].isna().sum() > 0 and df[col].notna().sum() > 0:
            df[col] = df[col].fillna(df[col].median())

    return df

In [ ]:
# Define function to clean extreme values
def remove_extreme_values(df):
    # Set limits for physiologically plausible values
    limits = {
        'SpO2': (60, 100),  # SpO2 range from 60% to 100%
        'HR': (30, 200),  # Heart rate from 30 to 200 bpm
        'PI': (0, 20),  # PI normally between 0 and 20
        'ETCO2': (0, 100),  # End-tidal CO2 normally 35-45 mmHg but allowing wider range
        'ETO2': (0, 100),  # End-tidal O2 percentage
        'ScalcO2': (0, 100),  # Calculated O2 percentage
        'RR': (4, 40)  # Respiratory rate from 4 to 40 breaths per minute
    }

    for col_type, (min_val, max_val) in limits.items():
        cols = [col for col in df.columns if col_type in col]
        for col in cols:
            if col in df.columns:
                # Replace values outside the limits with NaN
                df[col] = df[col].apply(lambda x: x if pd.isna(x) or (min_val <= x <= max_val) else np.nan)

    return df

In [ ]:
# Apply cleaning functions
df = remove_extreme_values(df)
df = fill_missing_values(df)

In [ ]:
# Drop columns with too many missing values and check results
threshold = 0.8 * len(df)
df = df.dropna(axis=1, thresh=threshold)

# Drop rows where all feature values are missing
df = df.dropna(how='all')

print("\nData shape after cleaning:", df.shape)
print("\nMissing values after cleaning:")
print(df.isnull().sum())

In [ ]:
# Define function to create device-specific features
def create_device_features(df):
    # Identify device columns
    devices = set([col.split('_')[0] for col in df.columns if '_' in col])

    for device in devices:
        device_cols = [col for col in df.columns if device in col]

        # Only process if we have enough columns for this device
        if len(device_cols) >= 2:
            # Calculate mean and std of SpO2 for this device if available
            spo2_col = [col for col in device_cols if 'SpO2' in col]
            if spo2_col and df[spo2_col[0]].notna().sum() > 0:
                df[f'{device}_SpO2_rolling_mean'] = df[spo2_col[0]].rolling(window=5, min_periods=1).mean()
                df[f'{device}_SpO2_rolling_std'] = df[spo2_col[0]].rolling(window=5, min_periods=1).std().fillna(0)

            # Calculate mean and std of HR for this device if available
            hr_col = [col for col in device_cols if 'HR' in col]
            if hr_col and df[hr_col[0]].notna().sum() > 0:
                df[f'{device}_HR_rolling_mean'] = df[hr_col[0]].rolling(window=5, min_periods=1).mean()
                df[f'{device}_HR_rolling_std'] = df[hr_col[0]].rolling(window=5, min_periods=1).std().fillna(0)

            # If we have both SpO2 and HR, create a ratio feature
            if spo2_col and hr_col and df[spo2_col[0]].notna().sum() > 0 and df[hr_col[0]].notna().sum() > 0:
                df[f'{device}_SpO2_HR_ratio'] = df[spo2_col[0]] / df[hr_col[0]]

    return df

In [ ]:
# Define function to create temporal features
def create_temporal_features(df):
    if 'Timestamp' in df.columns and pd.api.types.is_datetime64_dtype(df['Timestamp']):
        df['hour'] = df['Timestamp'].dt.hour
        df['minute'] = df['Timestamp'].dt.minute
        df['second'] = df['Timestamp'].dt.second

        # Create time-based features for physiological patterns
        df['time_of_day'] = df['hour'].apply(lambda x:
                                             'night' if 0 <= x < 6 else
                                             'morning' if 6 <= x < 12 else
                                             'afternoon' if 12 <= x < 18 else
                                             'evening')

        # One-hot encode time of day
        df = pd.get_dummies(df, columns=['time_of_day'], prefix='time')

    return df

In [ ]:
# Apply feature engineering functions
try:
    df = create_device_features(df)
except Exception as e:
    print(f"Warning in create_device_features: {e}")

try:
    df = create_temporal_features(df)
except Exception as e:
    print(f"Warning in create_temporal_features: {e}")

In [ ]:
# Create cross-device consistency features
devices_spo2 = [col for col in df.columns if
                'SpO2' in col and not col.endswith('_rolling_mean') and not col.endswith('_rolling_std')]
if len(devices_spo2) >= 2:
    # Calculate mean and std across devices
    df['cross_device_SpO2_mean'] = df[devices_spo2].mean(axis=1)
    df['cross_device_SpO2_std'] = df[devices_spo2].std(axis=1, ddof=0)

    # Calculate device deviation from mean
    for col in devices_spo2:
        df[f'{col}_dev_from_mean'] = df[col] - df['cross_device_SpO2_mean']

# Similarly for heart rate
devices_hr = [col for col in df.columns if
              '_HR' in col and not col.endswith('_rolling_mean') and not col.endswith('_rolling_std')]
if len(devices_hr) >= 2:
    df['cross_device_HR_mean'] = df[devices_hr].mean(axis=1)
    df['cross_device_HR_std'] = df[devices_hr].std(axis=1, ddof=0)

    for col in devices_hr:
        df[f'{col}_dev_from_mean'] = df[col] - df['cross_device_HR_mean']

print("\nFeatures after engineering:")
print(df.columns.tolist())
print("\nData shape after feature engineering:", df.shape)

In [ ]:
# Data preparation for modeling
# Drop non-feature columns
non_feature_cols = ['Timestamp', 'Sample']
feature_cols = [col for col in df.columns if col not in non_feature_cols]

# Handle remaining NaN values for modeling
X = df[feature_cols].fillna(df[feature_cols].median())

In [ ]:
# Define function to normalize data
def normalize_data(X, method='standard'):
    if method == 'standard':
        scaler = StandardScaler()
    elif method == 'robust':
        scaler = RobustScaler()
    else:
        raise ValueError("Invalid normalization method")

    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler


In [ ]:
#Apply normalization methods
X_standard, standard_scaler = normalize_data(X, method='standard')
X_robust, robust_scaler = normalize_data(X, method='robust')

In [ ]:
#Save the scalers
joblib.dump(standard_scaler, 'models/standard_scaler.pkl')
joblib.dump(robust_scaler, 'models/robust_scaler.pkl')

In [ ]:
#Dimension reduction for visualization
# Apply PCA for visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_standard)

plt.figure(figsize=(10, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
plt.title('PCA of Patient Data')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.savefig('results/pca_visualization.png')
plt.close()

In [ ]:
# Define function to evaluate and save models
def evaluate_and_save_model(name, model, X, X_pca=None):
    # Make predictions
    if hasattr(model, 'fit_predict'):
        y_pred = model.fit_predict(X)
    else:
        model.fit(X)
        y_pred = model.predict(X)

    # Convert predictions to outlier labels (assuming -1 or lower scores are outliers)
    if name == 'LOF' or name == 'IsolationForest':
        outliers = y_pred == -1
    else:  # For clustering-based methods
        outliers = y_pred == -1  # Adjust based on specific algorithm output

    # Calculate percentage of outliers
    outlier_percentage = 100 * outliers.sum() / len(outliers)

    # Save the model
    model_filename = f'models/{name}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pkl'
    joblib.dump(model, model_filename)

    # If we have PCA data, visualize the results
    if X_pca is not None:
        plt.figure(figsize=(10, 8))
        plt.scatter(X_pca[~outliers, 0], X_pca[~outliers, 1], c='blue', alpha=0.5, label='Inliers')
        plt.scatter(X_pca[outliers, 0], X_pca[outliers, 1], c='red', alpha=0.5, label='Outliers')
        plt.title(f'Outlier Detection using {name} (Outliers: {outlier_percentage:.2f}%)')
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.legend()
        plt.savefig(f'results/{name}_outliers.png')
        plt.close()

    # Try to calculate silhouette score if applicable (for clustering-based methods)
    sil_score = None
    if name in ['KMeans', 'DBSCAN'] and len(np.unique(y_pred)) > 1:
        try:
            sil_score = silhouette_score(X, y_pred)
        except:
            sil_score = "Not applicable"

    # Calculate some basic statistics on the outliers
    outlier_indices = np.where(outliers)[0]

    results = {
        'model_name': name,
        'outlier_percentage': outlier_percentage,
        'silhouette_score': sil_score,
        'outlier_indices': outlier_indices,
        'model_path': model_filename
    }

    return results

In [ ]:
# Initialize dictionary to store results
model_results = {}

In [ ]:
# K-Means Clustering
print("\nTraining K-Means clustering model...")
best_kmeans = None
best_score = -1
best_k = 2

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_standard)
    labels = kmeans.labels_

    # Try to calculate silhouette score
    try:
        score = silhouette_score(X_standard, labels)
        print(f"KMeans with {k} clusters - Silhouette Score: {score:.4f}")

        if score > best_score:
            best_score = score
            best_k = k
            best_kmeans = kmeans
    except:
        continue

if best_kmeans:
    # Add a cluster for outliers (could be the smallest or most distant cluster)
    labels = best_kmeans.labels_
    cluster_sizes = np.bincount(labels)
    smallest_cluster = np.argmin(cluster_sizes)

    # Mark the smallest cluster as outliers (-1)
    kmeans_outlier_labels = np.array(labels, copy=True)
    kmeans_outlier_labels[labels == smallest_cluster] = -1
    kmeans_outlier_labels[labels != smallest_cluster] = 1

    # Evaluate and save
    kmeans_results = evaluate_and_save_model('KMeans', best_kmeans, X_standard, X_pca)
    kmeans_results['best_k'] = best_k
    kmeans_results['silhouette_score'] = best_score
    model_results['KMeans'] = kmeans_results

In [ ]:
# DBSCAN Clustering
print("\nTraining DBSCAN model...")
best_dbscan = None
best_score = -1
best_eps = 0.5
best_min_samples = 5

param_grid = {
    'eps': [0.3, 0.5, 0.7, 1.0],
    'min_samples': [5, 10, 15, 20]
}

for eps in param_grid['eps']:
    for min_samples in param_grid['min_samples']:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_standard)

        # Skip if all points are classified as noise (-1)
        if np.all(labels == -1) or len(np.unique(labels)) == 1:
            continue

        # Try to calculate silhouette score
        try:
            score = silhouette_score(X_standard, labels)
            print(f"DBSCAN with eps={eps}, min_samples={min_samples} - Silhouette Score: {score:.4f}")

            if score > best_score:
                best_score = score
                best_eps = eps
                best_min_samples = min_samples
                best_dbscan = dbscan
        except:
            continue

if best_dbscan:
    # Evaluate and save
    dbscan_results = evaluate_and_save_model('DBSCAN', best_dbscan, X_standard, X_pca)
    dbscan_results['best_eps'] = best_eps
    dbscan_results['best_min_samples'] = best_min_samples
    dbscan_results['silhouette_score'] = best_score
    model_results['DBSCAN'] = dbscan_results

In [ ]:
# Isolation Forest
print("\nTraining Isolation Forest model...")
best_iso_forest = None
best_auc = -1
best_contamination = 0.1

for contamination in [0.01, 0.05, 0.1, 0.15]:
    iso_forest = IsolationForest(
        n_estimators=100,
        max_samples='auto',
        contamination=contamination,
        random_state=42
    )
    iso_forest.fit(X_standard)
    model_results[f'IsolationForest_{contamination}'] = evaluate_and_save_model(
        f'IsolationForest_{contamination}',
        iso_forest,
        X_standard,
        X_pca
    )


In [ ]:
#Local Outlier Factor
print("\nTraining Local Outlier Factor model...")
for n_neighbors in [5, 10, 20]:
    lof = LocalOutlierFactor(
        n_neighbors=n_neighbors,
        contamination=0.1
    )
    model_results[f'LOF_{n_neighbors}'] = evaluate_and_save_model(
        f'LOF_{n_neighbors}',
        lof,
        X_standard,
        X_pca
    )

In [ ]:
# Evaluate Feature Importance
# For IsolationForest, we can get feature importance
if 'IsolationForest_0.1' in model_results:
    iso_forest_model = joblib.load(model_results['IsolationForest_0.1']['model_path'])

    if hasattr(iso_forest_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': iso_forest_model.feature_importances_
        })

        feature_importance = feature_importance.sort_values('Importance', ascending=False)

        plt.figure(figsize=(10, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
        plt.title('Top 20 Features by Importance')
        plt.tight_layout()
        plt.savefig('results/feature_importance.png')
        plt.close()

In [ ]:
#Summary and Reporting
# Print model evaluation summary
print("\n=== MODEL EVALUATION SUMMARY ===")
for model_name, results in model_results.items():
    print(f"\nModel: {model_name}")
    print(f"Outlier Percentage: {results['outlier_percentage']:.2f}%")
    if results['silhouette_score'] is not None:
        print(f"Silhouette Score: {results['silhouette_score']}")
    print(f"Model saved to: {results['model_path']}")

In [ ]:
# Save summary to file
with open('results/model_summary.txt', 'w') as f:
    f.write("=== MODEL EVALUATION SUMMARY ===\n")
    for model_name, results in model_results.items():
        f.write(f"\nModel: {model_name}\n")
        f.write(f"Outlier Percentage: {results['outlier_percentage']:.2f}%\n")
        if results['silhouette_score'] is not None:
            f.write(f"Silhouette Score: {results['silhouette_score']}\n")
        f.write(f"Model saved to: {results['model_path']}\n")
        f.write(f"Number of outliers detected: {len(results['outlier_indices'])}\n")

print("\nAnalysis complete! Results saved to 'results' directory.")
print("Models saved to 'models' directory.")